In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV

In [2]:
df_train = pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/train.parquet")

In [3]:
Na = df_train.isna().sum()*100/(df_train.shape[0])
Na_cols = Na.sort_values(ascending = False).head(25).index
df_train.drop(columns = Na_cols, inplace = True)
del Na
cat_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
nonsig_cols = ["D_65", "S_23", "D_135", "S_24", "B_26", "D_143", "B_40", "D_141", "B_19",
              "S_22", "D_137", "R_7", "D_71", "B_2", "B_27", "D_69", "R_20", "D_83", "D_75",
              "B_36", "R_25", "D_139", "R_28", "R_10", "R_24", "D_119", "R_4", "B_10"]

In [4]:
y = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
df_train = pd.merge(df_train, y, on = "customer_ID")
df_train = df_train.groupby("customer_ID").head(2)
df_train = df_train.fillna(method = "ffill")
df_date = df_train.pop("S_2")
del df_date
df_train = df_train.set_index("customer_ID", drop = True).sort_index()


In [5]:
y = df_train.pop("target")
features = list(set(df_train.columns) - set(cat_col) - set(nonsig_cols)) 
X = df_train[features]
cat_dummies = pd.get_dummies(df_train[cat_col])
X = pd.concat([X, cat_dummies], axis = "columns")
del df_train

In [6]:
#model = LogisticRegression()
#model.fit(X,y)
#model.score(X,y)
#import statsmodels.api as sm
#model = sm.Logit(y, X).fit()
#print(model.summary())
#from sklearn.naive_bayes import GaussianNB
#gnb = GaussianNB()
#gnb.fit(X, y)

In [7]:
model = LogisticRegressionCV(cv = 5, random_state = 42).fit(X,y)
#model.score(X,y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [8]:
del X
del y

In [9]:
test_df = (pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/test.parquet")
            .groupby("customer_ID")
           .tail(1)
           .fillna(method = "ffill")
           .set_index("customer_ID", drop = True)
           .sort_index())

In [10]:
cat_cols = pd.get_dummies(test_df[cat_col])
test_df = pd.concat([test_df[features], cat_cols], axis = "columns")

In [11]:
y_pred = model.predict_proba(test_df)
del test_df
sub = pd.read_csv("../input/amex-default-prediction/sample_submission.csv")
sub = sub.sort_values("customer_ID")

In [12]:
sub["prediction"] = y_pred[:, 1]
sub.to_csv("submission.csv", index = False)